# Serialization

```{autolink-concat}
```

In [ ]:
from __future__ import annotations

import json
import logging
import math
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import Markdown, display
from scipy.interpolate import griddata, interp2d
from tensorwaves.function.sympy import create_function
from tqdm.auto import tqdm

from polarimetry import formulate_polarimetry
from polarimetry.data import (
    create_data_transformer,
    generate_meshgrid_sample,
    generate_phasespace_sample,
)
from polarimetry.io import (
    export_polarimetry_field,
    import_polarimetry_field,
    mute_jax_warnings,
    perform_cached_doit,
)
from polarimetry.lhcb import load_model_builder, load_model_parameters
from polarimetry.lhcb.particle import load_particles
from polarimetry.plot import use_mpl_latex_fonts

logging.getLogger().setLevel(logging.ERROR)
mute_jax_warnings()

model_choice = "Default amplitude model"
model_file = "../../data/model-definitions.yaml"
particles = load_particles("../../data/particle-definitions.yaml")
amplitude_builder = load_model_builder(model_file, particles, model_choice)
imported_parameter_values = load_model_parameters(
    model_file, amplitude_builder.decay, model_choice
)
reference_subsystem = 1
model = amplitude_builder.formulate(reference_subsystem)
model.parameter_defaults.update(imported_parameter_values)

NO_TQDM = "EXECUTE_NB" in os.environ
if NO_TQDM:
    logging.getLogger().setLevel(logging.ERROR)

In [ ]:
polarimetry_exprs = formulate_polarimetry(amplitude_builder, reference_subsystem)
unfolded_exprs = [
    perform_cached_doit(expr.doit().xreplace(model.amplitudes))
    for expr in tqdm(
        [model.full_expression, *polarimetry_exprs], disable=NO_TQDM, leave=False
    )
]
actual_funcs = [
    create_function(expr.xreplace(model.parameter_defaults), backend="jax")
    for expr in tqdm(
        unfolded_exprs, leave=False, desc="Lambdifying", disable=NO_TQDM
    )
]

In [ ]:
resolution = 100
transformer = create_data_transformer(model)
grid_sample = generate_meshgrid_sample(model.decay, resolution)
grid_sample = transformer(grid_sample)
X = grid_sample["sigma1"]
Y = grid_sample["sigma2"]

In [ ]:
alpha_x_func = actual_funcs[1]
alpha_x = alpha_x_func(grid_sample).real
df = pd.DataFrame(alpha_x, index=X[0], columns=Y[:, 0])
df.to_json("alpha-x-pandas.json")
df.to_json("alpha-x-pandas-json.zip", compression={"method": "zip"})
df.to_csv("alpha-x-pandas.csv")

df_dict = df.to_dict()
filtered_df_dict = {
    x: {y: v for y, v in row.items() if not math.isnan(v)}
    for x, row in df_dict.items()
}
with open("alpha-x-python.json", "w") as f:
    json.dump(filtered_df_dict, f)

json_dict = dict(
    x=X[0].tolist(),
    y=Y[:, 0].tolist(),
    z=alpha_x.tolist(),
)
with open("alpha-x-arrays.json", "w") as f:
    json.dump(json_dict, f, separators=(",", ":"))

In [ ]:
def render_kilobytes(path, markdown: bool = False) -> str:
    byt = os.path.getsize(path)
    kb = f"{1e-3*byt:.0f}"
    if markdown:
        return f"\n - **{kb} kB**: {{download}}`{path}`"
    return f"\n  {kb:>4s} kB  {path}"


src = f"File sizes for {len(X[0])}x{len(Y[:, 0])} grid:"
markdown = "EXECUTE_NB" in os.environ
src += render_kilobytes("alpha-x-arrays.json", markdown)
src += render_kilobytes("alpha-x-pandas.json", markdown)
src += render_kilobytes("alpha-x-python.json", markdown)
src += render_kilobytes("alpha-x-pandas-json.zip", markdown)
src += render_kilobytes("alpha-x-pandas.csv", markdown)
if markdown:
    display(Markdown(src))
else:
    print(src)

## Exported polarimetry grids

Decided to use the `alpha-x-arrays.json` format. It can be exported with {func}`.export_polarimetry_field`.

In [ ]:
filename = "polarimetry-model-0.json"
export_polarimetry_field(
    sigma1=X[0],
    sigma2=Y[:, 0],
    intensity=actual_funcs[0](grid_sample).real,
    alpha_x=actual_funcs[1](grid_sample).real,
    alpha_y=actual_funcs[2](grid_sample).real,
    alpha_z=actual_funcs[3](grid_sample).real,
    filename=filename,
    metadata={"model description": model_choice},
)

In [ ]:
if "EXECUTE_NB" in os.environ:
    byt = os.path.getsize(filename)
    kb = f"{1e-3*byt:.0f}"
    src = (
        f"Polarimetry grid can be downloaded here: {{download}}`{filename}`"
        f" ({kb} kB)."
    )
    display(Markdown(src))

## Import and interpolate

::::{margin}
:::{warning}
{obj}`~numpy.nan` values have to be replaced with `0.0` using {func}`numpy.nan_to_num`.
:::
::::

In [ ]:
field_definition = import_polarimetry_field("polarimetry-model-0.json")
imported_sigma1 = field_definition["m^2_Kpi"]
imported_sigma2 = field_definition["m^2_pK"]
imported_arrays = (
    field_definition["intensity"],
    field_definition["alpha_x"],
    field_definition["alpha_y"],
    field_definition["alpha_z"],
)
interpolated_funcs = tuple(
    interp2d(imported_sigma1, imported_sigma2, np.nan_to_num(z), kind="linear")
    for z in imported_arrays
)

In [ ]:
interpolated_funcs[1](0.8, 3.6)

Use {obj}`numpy.vectorize` to compute the interpolated values over a random phase space sample:

In [ ]:
n_points = 10_000
mini_sample = generate_phasespace_sample(model.decay, n_points, seed=0)
mini_sample = transformer(mini_sample)
x = mini_sample["sigma1"]
y = mini_sample["sigma2"]
z_interpolated = [
    np.vectorize(func)(x, y) for func in tqdm(interpolated_funcs, disable=NO_TQDM)
]
z_interpolated[0]

In [ ]:
use_mpl_latex_fonts()
plt.rc("font", size=18)
fig, axes = plt.subplots(
    dpi=200,
    figsize=(15, 11.5),
    gridspec_kw={"width_ratios": [1, 1, 1, 1.2]},
    ncols=4,
    nrows=3,
    sharex=True,
    sharey=True,
)
plt.subplots_adjust(hspace=0.1, wspace=0.03)
fig.suptitle("Comparison interpolated and actual values", y=0.94)

points = np.transpose([x, y])
for i in tqdm(range(4), disable=NO_TQDM, leave=False):
    if i == 0:
        title = "$I$"
        cmap = plt.cm.viridis
        clim = None
    else:
        title = Rf"$\alpha_{'xyz'[i-1]}$"
        cmap = plt.cm.coolwarm
        clim = (-1, +1)
    axes[0, i].set_title(title)

    z_actual = actual_funcs[i](mini_sample)
    z_diff = 100 * ((z_interpolated[i] - z_actual) / z_actual).real
    Z_interpolated = griddata(points, z_interpolated[i], (X, Y))
    Z_diff = griddata(points, z_diff, (X, Y))

    mesh = (
        axes[0, i].pcolormesh(X, Y, Z_diff, clim=(-100, +100), cmap=plt.cm.coolwarm),
        axes[1, i].pcolormesh(X, Y, Z_interpolated, cmap=cmap),
        axes[2, i].pcolormesh(X, Y, actual_funcs[i](grid_sample).real, cmap=cmap),
    )
    if i != 0:
        mesh[1].set_clim(-1, +1)
        mesh[2].set_clim(-1, +1)
    if i == 3:
        c_bar = [fig.colorbar(mesh[j], ax=axes[j, i]) for j in range(3)]
        c_bar[0].ax.set_ylabel(R"Difference (\%)")
        c_bar[1].ax.set_ylabel("Interpolated distribution")
        c_bar[2].ax.set_ylabel("Original distribution")
        for c in c_bar[1:]:
            c.ax.set_yticks([-1, 0, +1])
            c.ax.set_yticklabels(["-1", "0", "+1"])
plt.show()

:::{note}
The interpolated values over this phase space sample have been visualized by interpolating again over a {obj}`~numpy.meshgrid` with {obj}`scipy.interpolate.griddata`.
:::